In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from gensim.models import KeyedVectors
tqdm.pandas()
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity

Utiliza o GLOVE em PT-BR
http://nilc.icmc.usp.br/nilc/index.php/repositorio-de-word-embeddings-do-nilc

In [2]:
glove = KeyedVectors.load_word2vec_format('./glove_s50.txt')

In [3]:
df = pd.read_excel("Disciplinas.xlsx")

In [4]:
df.Disciplinas = df.Disciplinas.str.lower()

In [5]:
len(glove.index_to_key)

929605

In [6]:
# dir(glove)

In [7]:
df["embedding"] = \
    df.Disciplinas.progress_apply(lambda x: np.array([glove[x] for x in x.split() if x in glove.index_to_key]).max(axis=0))

100%|███████████████████████████████████████████████████████████████████████████████| 300/300 [00:00<00:00, 872.85it/s]


In [8]:
df['_key'] = 1

In [9]:
result = pd.merge(df, df, on='_key', how='outer').drop('_key', axis=1)

In [10]:
result["cosine_distance"] = result.progress_apply(lambda x: 
    cosine_distances(x["embedding_x"].reshape(1, -1), x["embedding_y"].reshape(1, -1))[0][0], axis=1)

100%|██████████████████████████████████████████████████████████████████████████| 90000/90000 [00:48<00:00, 1864.61it/s]


In [11]:
result["cosine_similarity"] = result.progress_apply(lambda x: 
    cosine_similarity(x["embedding_x"].reshape(1, -1), x["embedding_y"].reshape(1, -1))[0][0], axis=1)

100%|██████████████████████████████████████████████████████████████████████████| 90000/90000 [00:46<00:00, 1940.71it/s]


In [12]:
result.drop(["embedding_x", "embedding_y"], axis=1).to_excel("teste3.xlsx", index=False)